# 1. 批量下载

In [ ]:
from shreport import SH

cookies = {"Cookie": '您的cookies'}
sh = SH(cookies)
sdf = sh.companys()

#将查询结果存储
#sdf.to_excel('上证交易所上市公司名录.xlsx')

#显示前5条数据
sdf.head()

In [ ]:
from pathlib import Path
from shreport import SH

cookies = {"Cookie": 'sseMenuSpecial=8311; yfx_c_g_u_id_10000042=_ck22041420474316030190275754320; yfx_f_l_v_t_10000042=f_t_1649940463492__r_t_1649940463492__v_t_1649940463492__r_c_0; VISITED_MENU=%5B%228312%22%5D'}
sh = SH(cookies)
#获取当前代码所在的文件夹路径
cwd = Path().cwd() 
#以浦发银行为例股票代码600000
sh.download(code='600004', savepath=cwd)

# 2. 整理md&a

In [1]:
import pandas as pd

df = pd.read_excel('small_MD_A.xlsx')
df.head()

,Scode,Coname,Year,Anatime,BusDA
0,600136,当代明诚,2005,2005-06-30,1、公司主营业务经营情况说明\n\n\n本报告期公司主营业务为：房地产开发与商品房销售、电子...
1,861,海印股份,2018,2018-12-31,第四节 经营情况讨论与分析\n\n一、概述\n\n2018 年，随着经济结构不断优化，居民人...
2,600759,洲际油气,2016,2016-06-30,"截止本报告期末,公司实现营业收入559,628,476.62元,较上年同期下降14.35%;..."
3,600988,赤峰黄金,2012,2012-12-31,第三节 董事会报告\n\n2012 年，公司实施了重大资产重组，公司资产结构及主营业务发生了...
4,600804,鹏博士,2007,2007-12-31,"（一）管理层讨论与分析\n\n\n报告期内,公司顺利完成非公开发行股票及重大资产收购工作,使..."


In [2]:
print("A股  :{} 家".format(len(df.Coname.unique())))
print()
print("年份  :{} ".format(sorted(df.Year.unique())))
print()
print("报告数: {} ".format(len(df)))

A股  :2742 家

年份  :[1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020] 

报告数: 5000 


# 3. 短视主义词典

In [4]:

short_terms_dicts = {'短视': ['天内', '数月', '年内', '年底', '尽快',  '立刻', '马上']}



# 4. 短视主义计算

In [ ]:
!pip3 install cntext==1.6

In [6]:
from cntext import sentiment
help(sentiment)

Help on function sentiment in module cntext.stats:

sentiment(text, diction, language='chinese')
    使用diy词典进行情感分析，计算各个情绪词出现次数，未考虑强度副词、否定词对情感的复杂影响，
    :param text:  待分析中文文本
    :param diction:  情感词字典；
    :param language: 语言类型，"chinese"或"english"，默认"chinese"
    
    {'category1':  'category1 词语列表',
     'category2': 'category2词语列表',
     'category3': 'category3词语列表',
     ...
    }
    
    :return:



In [11]:
from cntext import sentiment

    
def short_term_infos(text, diction):
    return pd.Series(sentiment(text=text, 
                               diction=diction))

text = '我公司需要尽快落实新战略，争取月底转型升级，力争年底上市。'


sentiment(text=text, 
          diction=short_terms_dicts)

{'短视_num': 3, 'stopword_num': 4, 'sentence_num': 1, 'word_num': 17}

In [12]:
short_df = df['BusDA'].apply(short_term_infos, 
                             diction=short_terms_dicts)

new_df = pd.concat([df, short_df], axis=1)

new_df['短视主义程度']=new_df['短视_num']/new_df['word_num']

new_df[new_df['短视主义程度']>0]

,Scode,Coname,Year,Anatime,BusDA,短视_num,stopword_num,sentence_num,word_num,短视主义程度
3,600988,赤峰黄金,2012,2012-12-31,第三节 董事会报告\n\n2012 年，公司实施了重大资产重组，公司资产结构及主营业务发生了...,1,1281,281,4566,0.000219
4,600804,鹏博士,2007,2007-12-31,"（一）管理层讨论与分析\n\n\n报告期内,公司顺利完成非公开发行股票及重大资产收购工作,使...",1,706,110,2881,0.000347
6,600227,赤天化,2009,2009-06-30,（一） 报告期内整体经营情况的讨论与分析报告期内，受全球金融危机蔓延的影响，公司面临的外部经...,1,333,60,1334,0.000750
12,300183,东软载波,2012,2012-06-30,2012年上半年，随着国家发改委提出加快阶梯电价实施进程，国网公司加快了电力用户信息采集系统...,7,196,26,750,0.009333
15,2175,东方网络,2018,2018-12-31,第四节 经营情况讨论与分析\n\n一、 概述\n\n2018 年是公司转型“文化传媒行业”的...,3,953,677,8297,0.000362
...,...,...,...,...,...,...,...,...,...,...
4983,600509,天富能源,2009,2009-06-30,（一） 报告期内整体经营情况的讨论与分析报告期内，公司电、热及天然气的产销量都较上年同期有较...,1,231,41,798,0.001253
4985,536,华映科技,2004,2004-12-31,一、经营情况的讨论与分析\n\n\n报告期内，公司属于机械工业的中小型电机行业，仍以电机、房...,1,212,30,655,0.001527
4987,408,藏格控股,2008,2008-12-31,一、报告期内公司经营情况回顾\n\n\n2008 年是公司生产经营较为艰难的一年。全年生产经...,1,295,48,977,0.001024
4988,550,江铃汽车,2004,2004-12-31,（一）经营情况\n\n\n本公司的主要业务是生产和销售轻型汽车以及相关的零部件。主要产品包括...,1,278,37,764,0.001309


In [13]:
def getvec(word):
    return w2v_model.get_vector(word)

v1 = getvec('班子')
v1

array([-1.6928811e-02,  3.7603001e-03,  7.7051586e-03, -8.0923783e-03,
       -2.2519925e-03,  1.1721392e-03, -7.7138282e-03,  1.2574076e-02,
        1.6803989e-02,  2.0844387e-03,  7.8222575e-04, -3.9842480e-04,
       -1.2934345e-02, -4.6401178e-03, -1.2668942e-02,  6.4697088e-04,
       -1.4708638e-02,  1.6241480e-03,  1.0422479e-03, -2.2304073e-02,
        2.1653239e-02,  1.1029784e-02,  7.4102250e-03, -1.7052751e-02,
        5.0417199e-03,  1.5559165e-02, -1.9240506e-03,  9.0795005e-04,
       -3.4530319e-02, -3.0115025e-02, -6.6113039e-03,  2.3094827e-02,
       -3.2706219e-03, -8.8423630e-03,  3.9391471e-03, -6.7869779e-03,
        1.0314009e-02,  4.6877684e-03, -3.6454117e-03, -1.5133144e-02,
       -1.0863796e-02, -3.2419243e-03, -9.3616368e-03,  7.7638794e-03,
        1.5916402e-02,  5.1452004e-04, -1.2945516e-02, -4.4644321e-03,
        1.6014639e-02,  2.2467042e-03,  9.3664844e-03, -3.0118048e-03,
       -5.7858834e-03, -4.3510059e-03, -3.0939199e-03, -1.8782240e-03,
      

In [31]:
female = (getvec('女性') + getvec('女') + getvec('妈妈'))/3
female

array([-9.7802642e-04, -1.0850709e-03,  5.5397180e-04, -2.3114888e-03,
       -3.2688782e-03, -2.9677464e-04,  1.3205163e-03,  1.5658514e-03,
        1.2011837e-03, -2.8580569e-03,  6.7808077e-04,  7.9690712e-03,
        4.0947358e-04,  3.4272671e-04,  9.1581093e-03, -4.4239489e-03,
        1.8016202e-03, -3.1368432e-03,  7.7428832e-03, -4.9203518e-03,
       -1.2628173e-03, -2.4986840e-03,  2.5500045e-03,  1.1967588e-03,
       -1.6034986e-03,  3.0349717e-03, -3.5297943e-03,  2.5465956e-03,
        2.9796603e-04, -1.1809610e-03,  3.0079009e-03, -2.5266767e-03,
       -2.3871132e-03,  6.7020305e-03, -1.5548551e-03,  2.4140074e-03,
       -1.6895899e-03,  5.3502696e-05, -6.9087720e-03,  2.0606772e-03,
        2.1241449e-03, -6.3858684e-03, -2.7876145e-03, -2.1434324e-03,
       -1.4681182e-03, -1.5298518e-03, -1.3644465e-04,  8.2780374e-04,
        2.8159006e-03, -4.0065390e-03,  1.9975745e-03,  3.3509952e-03,
        6.9097505e-04,  7.8044818e-03,  8.0248370e-04,  1.0167925e-03,
      

In [32]:
male = (getvec('男性') + getvec('男') + getvec('爸爸'))/3
male

array([-4.3988246e-03,  4.8705484e-04, -5.9010778e-03,  6.2376154e-03,
       -3.0389763e-03,  2.4262411e-03, -5.7574692e-03,  3.2243847e-03,
       -1.6693409e-03, -4.8531652e-03, -1.8645826e-03,  3.0020718e-04,
        2.4983359e-03,  1.5611370e-03, -3.5955273e-03,  3.9251926e-03,
        3.4007325e-03,  1.7652899e-03, -1.2461996e-03, -1.0120933e-03,
       -2.6910265e-03, -1.2663738e-03, -5.6406576e-04,  1.6866199e-03,
        2.9088215e-03, -8.2984036e-03,  2.1232932e-03,  1.7734448e-04,
        4.5281863e-03,  4.4056526e-03,  1.5914425e-03,  3.3137240e-03,
       -2.9367106e-03,  4.3961504e-03,  5.6690336e-03,  1.2382437e-03,
       -2.7088635e-04, -2.5352915e-03, -5.3167902e-03,  2.2760502e-03,
        4.7371318e-04, -1.0607719e-03, -1.3922054e-04, -2.0893861e-03,
       -2.8699785e-03, -2.3632746e-03, -3.4749014e-03,  3.8159664e-03,
       -3.3181971e-03,  1.4429330e-03,  1.1258951e-03,  3.6877200e-03,
       -6.0419895e-04, -1.0684865e-03, -3.9230301e-03, -1.8448448e-03,
      

In [35]:
w2v_model.cosine_similarities(getvec('工程师'), [getvec('男'), 
                                                getvec('女')])

array([-0.06231755, -0.09240822], dtype=float32)

In [43]:
w2v_model.cosine_similarities(getvec('风险'), [getvec('危机'), 
                                             getvec('妈妈')])

array([0.9833715 , 0.12054411], dtype=float32)

In [44]:
w2v_model.cosine_similarities(getvec('管控'), [getvec('风险'), 
                                             getvec('妈妈')])

array([0.0811202 , 0.21992126], dtype=float32)

In [54]:
w2v_model.similar_by_key('不良资产')

[('风险', 0.9991183280944824),
 ('业务', 0.9990758299827576),
 ('本行', 0.9990417957305908),
 ('资本', 0.9990257620811462),
 ('“', 0.999019980430603),
 ('管理', 0.9989909529685974),
 ('提高', 0.9989698529243469),
 (',', 0.9989678859710693),
 ('经营', 0.9989630579948425),
 ('市场', 0.9989481568336487)]

In [73]:
w2v_model.similar_by_key('发行')

[('业务', 0.9997439980506897),
 (',', 0.9996848702430725),
 ('提高', 0.9996770024299622),
 ('本行', 0.9996767640113831),
 ('零售', 0.9996206760406494),
 ('风险', 0.9996152520179749),
 ('商业银行', 0.9996132850646973),
 ('各项', 0.999573826789856),
 ('存款', 0.999567985534668),
 ('资产', 0.99955815076828)]

In [60]:
w2v_model.similar_by_key('拨备')

[('NUM', 0.9913316369056702),
 ('增长', 0.991309404373169),
 ('期末', 0.9912890791893005),
 ('利润', 0.9912500977516174),
 ('增加', 0.9912283420562744),
 ('%', 0.9911730289459229),
 ('增幅', 0.9911189675331116),
 ('收入', 0.9910915493965149),
 ('贷款', 0.9910189509391785),
 ('保持', 0.9909676909446716)]

In [61]:
w2v_model.similar_by_key('利润')

[(',', 0.9993423223495483),
 ('本行', 0.9993217587471008),
 ('贷款', 0.9993171095848083),
 ('全行', 0.9993003010749817),
 ('各项', 0.9992858171463013),
 ('资产', 0.9992812275886536),
 ('NUM', 0.9992715120315552),
 ('存款', 0.9992536902427673),
 ('资本', 0.9992210268974304),
 ('人民币', 0.9992175698280334)]

In [2]:
from cntext import W2VModels
import os

#初始化模型
model = W2VModels(cwd=os.getcwd(), lang='chinese')  #语料数据 w2v_corpus.txt
model.train(input_txt_file='data/三体.txt')


#根据种子词，筛选出没类词最相近的前100个词
model.find(seedword_txt_file='data/santi_seed.txt', 
           topn=100)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/sc/3mnt5tgs419_hk7s16gq61p80000gn/T/jieba.cache


Step 1/4:...预处理    语料 ...


Loading model cost 0.515 seconds.
Prefix dict has been built successfully.


Step 2/4:...训练   word2vec模型 ...
Step 3/4:...准备 每个seed在word2vec模型中的相似候选词...
Step 4/4 完成! 耗时 8 s
